In [1]:
import pandas as pd
from common.data import DataLoader
from common.metrics import map_at_k
from models.popular import PopularRecommender, SegmentRecommender
from models.lightfm import WeightFeaturedLightFM
from recsys_course.const import *

/Users/d-emeljanov/pet_projects/recsys-course/notebooks/venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = DataLoader.from_folder(
    '../data/preprocessed/',
    user_col='user_id',
    item_col='item_id',
    date_col='last_watch_dt',
    watched_pct_min=0
)

In [3]:
train, test = data.get_train_test(test_size=0.3)

train[DATE_COL] = pd.to_datetime(train[DATE_COL])
test[DATE_COL] = pd.to_datetime(test[DATE_COL])

In [4]:
cold_user_ids = set(test['user_id'].unique().tolist()).difference(set(train['user_id'].unique().tolist()))

test_cold = test[test['user_id'].isin(cold_user_ids)]
test_warm = test[~test['user_id'].isin(cold_user_ids)]

## Cold Predictions

In [5]:
test_cold_df = test_cold.groupby('user_id')['item_id'].apply(list).reset_index().rename(columns={'item_id': 'real'})

In [6]:
popular = PopularRecommender(
    fb__min_watched_pct=10,
    fb__total_dur_min=2000,
    days=10,
    date_col=DATE_COL,
    user_col=USER_COL,
    item_col=ITEM_COL
)

popular.fit(train)

In [7]:
test_cold_df['recs'] = popular.recommend(test_cold_df['user_id'].tolist(), 10)

In [8]:
map_at_k(
    10,
    test_cold_df['recs'],
    test_cold_df['real']
)

0.27165462904601767

## Warm Predictions

### CF

In [73]:
model = SegmentRecommender(
    days=10,
    date_col=DATE_COL,
    user_col=USER_COL,
    item_col=ITEM_COL,
    fb__min_watched_pct=10,
    fb__total_dur_min=2000,
    segment=['age', 'sex']
)

model.add_user_features(data.users)

model.fit(train)

test_warm_df['recs_test'] = model.recommend(test_warm_df['user_id'].tolist(), 200)

test_warm_df['perc'] = test_warm_df.apply(lambda x: len(list(set(x['real']).intersection(set(x['recs_test'])))) / len(x['real']), axis=1)
test_warm_df['perc'].mean(), test_warm_df['perc'].quantile(0.25)

32it [00:05,  5.57it/s]


(0.4521586103305911, 0.0)

'Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких, флешбек, видения, несчастный, случай, разговоры, любовники, кома'

In [1]:
from catboost.text_processing import Tokenizer

w = data.items.loc[0].keywords

token = Tokenizer(
    delimiter=', ',
    lowercasing=True,
    lemmatizing=True
)

token.tokenize(w)

NameError: name 'data' is not defined

In [9]:
lfm = WeightFeaturedLightFM(
    notseen_watched_upper=95,
    notseen_watched_lower=5,
    no_components=150,
    date_col=DATE_COL,
    user_col=USER_COL,
    item_col=ITEM_COL,
    user_features_col=None,
    item_features_col=None,
    preprocess_array_split=None
)

lfm.fit(train)

In [10]:
test_warm_df = test_warm.groupby('user_id')['item_id'].apply(list).reset_index().rename(columns={'item_id': 'real'})

In [11]:
test_warm_df['recs'] = lfm.recommend(test_warm_df['user_id'].tolist(), 50)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191361/191361 [15:36<00:00, 204.25it/s]


In [ ]:
map_at_k(
    10,
    test_warm_df['recs'],
    test_warm_df['real']
)

### Catboost

In [ ]:
!pip install -q catboost

In [ ]:
test_warm_df['recs'] = lfm.recommend(test_warm_df['user_id'].tolist(), 100)

 77%|██████████████████████████████████████████████████████████████████████████████████████████████▏                            | 146545/191361 [12:01<03:51, 193.79it/s]

In [12]:
cbc_df = train.loc[:, ['user_id', 'item_id']]
cbc_df.loc[:, 'y'] = 1
cbc_unused = data.unused.copy()
cbc_unused['y'] = 0

cbc_df = pd.merge(
    left=pd.merge(
        left=pd.concat([cbc_df, cbc_unused]),
        right=data.users,
        on=[USER_COL],
        how='left'
    ),
    right=data.items,
    on=[ITEM_COL],
    how='left'
)

cbc_df

,user_id,item_id,y,age,income,sex,kids_flg,content_type,title,title_orig,...,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat,for_kids_rating
0,1067648,2358,1,age_55_64,income_20_40,zh,0.0,film,нимфоманка. фильм первый,Nymphomaniac: Vol. I,...,"бельгия, германия, дания, франция",-1,18.0,unknown,ларс фон триер,"Стеллан Скарсгард, Шайа ЛаБаф, Ума Турман, Йен...",История эротических переживаний женщины от рож...,"нахлыст, девственница, униформа, грех, нимфома...",2010-2020,0
1,735886,9935,1,age_18_24,income_40_60,zh,1.0,film,антебеллум,Antebellum,...,сша,-1,18.0,unknown,"джерард буш, кристофер ренц","Жанель Моне, Эрик Ланж, Джек Хьюстон, Кирси Кл...","Известная писательница, активистка и борец за ...","2020, соединенные штаты, антебеллум",2020_inf,0
2,993358,7417,1,age_25_34,income_40_60,m,0.0,film,стендап под прикрытием,Undercover standup,...,россия,-1,16.0,unknown,олег асадулин,"Валентина Мазунина, Кирилл Нагиев, Зоя Бербер,...",Дерзкая и циничная опер в юбке Светлана Артюхо...,"2020, россия, стендап, под, прикрытием",2020_inf,0
3,429759,334,1,age_35_44,income_60_90,zh,0.0,film,храбрая сердцем,Brave,...,сша,-1,6.0,unknown,"марк эндрюс, бренда чепмен, стив пёрселл","Келли Макдоналд, Эмма Томпсон, Билли Коннолли,...",Испокон веков мифы и легенды окутывают загадоч...,"Шотландия, бунтарь, храбрость, королевство, лу...",2010-2020,1
4,506639,4141,1,age_18_24,income_20_40,m,0.0,film,пятьдесят оттенков серого,Fifty Shades of Grey,...,сша,-1,18.0,unknown,сэм тейлор-джонсон,"Виктор Расук, Дакота Джонсон, Джейми Дорнан, Д...","Анастейша Стил — скромная студентка, живущая в...","на основе романа или книги, извращение, порка,...",2010-2020,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13455161,1097557,12597,0,age_35_44,income_20_40,zh,0.0,film,эволюция борна,The Bourne Legacy,...,"сша, япония",-1,16.0,unknown,тони гилрой,"Джереми Реннер, Джоан Аллен, Донна Мерфи, Дэви...",Параллельно с программой ЦРУ по подготовке кил...,"убийца, волк, сеул, мэриленд, тайная операция,...",2010-2020,0
13455162,1097557,9332,0,age_35_44,income_20_40,zh,0.0,film,заноза,Splinter,...,сша,-1,18.0,unknown,тоби уилкинс,"Джилл Вагнер, Лорел Уитсетт, Паоло Костанзо, Р...",Молодая пара оказывается заперта на заброшенно...,"Заноза, 2008, США, притяжение, противоположнос...",2000-2010,0
13455163,1097557,11352,0,age_35_44,income_20_40,zh,0.0,film,тёмная сторона,Virtualia Episode Three: Dark Side,...,швеция,-1,21.0,unknown,антонио адамо,"Линн Стоун, Клаудия Риччи, Софи Эванс, Сандра ...",Приключенческая эротика в духе Индианы Джонса ...,", Анальный секс, Банкомат, Взгляд в камеру, Вт...",2000-2010,0
13455164,1097557,15512,0,age_35_44,income_20_40,zh,0.0,series,в последний раз прощаюсь,None,...,украина,-1,16.0,unknown,андрей черных,"Александра Польгуй, Андрей Романий, Андрей Фед...",Когда-то в молодости Андрей спас тонущую девоч...,"последний, раз, прощаюсь, 2017, Украина, безот...",2010-2020,0


In [53]:
cbc_df.columns

Index(['user_id', 'item_id', 'y', 'age', 'income', 'sex', 'kids_flg',
       'content_type', 'title', 'title_orig', 'release_year', 'genres',
       'countries', 'for_kids', 'age_rating', 'studios', 'directors', 'actors',
       'description', 'keywords', 'release_year_cat', 'for_kids_rating'],
      dtype='object')

In [54]:
cbc_df['genres']

0                                               драмы
1                                     драмы, триллеры
2                                             комедии
3           мультфильм, фэнтези, приключения, комедии
4                                           мелодрамы
                              ...                    
13455161               боевики, триллеры, приключения
13455162      фантастика, зарубежные, триллеры, ужасы
13455163                                 для взрослых
13455164                        зарубежные, мелодрамы
13455165                           советские, военные
Name: genres, Length: 13455166, dtype: object

In [13]:
cbc_df['sex'] = cbc_df['sex'].fillna('unknown')
cbc_df['age'] = cbc_df['age'].fillna('age_unknown')
cbc_df['genres'] = cbc_df['genres'].fillna('genres_unknown')

In [14]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(
    cat_features=['sex', 'age', 'release_year_cat', 'content_type'],
    text_features=['genres'],
    iterations=30
)

cbc.fit(cbc_df[['sex', 'age', 'release_year_cat', 'content_type', 'genres']], cbc_df['y'])

Learning rate set to 0.5
0:	learn: 0.5713483	total: 3.38s	remaining: 1m 38s
1:	learn: 0.5381965	total: 6.92s	remaining: 1m 36s
2:	learn: 0.5013839	total: 10s	remaining: 1m 30s
3:	learn: 0.4593684	total: 13.5s	remaining: 1m 27s
4:	learn: 0.4382221	total: 17.2s	remaining: 1m 25s
5:	learn: 0.4233655	total: 20.2s	remaining: 1m 20s
6:	learn: 0.4178412	total: 23.1s	remaining: 1m 15s
7:	learn: 0.4125235	total: 26.2s	remaining: 1m 12s
8:	learn: 0.4102451	total: 28.7s	remaining: 1m 7s
9:	learn: 0.4082438	total: 31.8s	remaining: 1m 3s
10:	learn: 0.4045613	total: 35s	remaining: 1m
11:	learn: 0.3999742	total: 38s	remaining: 57s
12:	learn: 0.3981588	total: 40.4s	remaining: 52.8s
13:	learn: 0.3957874	total: 42.8s	remaining: 48.9s
14:	learn: 0.3940365	total: 45.3s	remaining: 45.3s
15:	learn: 0.3928343	total: 47.6s	remaining: 41.7s
16:	learn: 0.3921093	total: 50.1s	remaining: 38.3s
17:	learn: 0.3912654	total: 52.6s	remaining: 35.1s
18:	learn: 0.3896875	total: 55.6s	remaining: 32.2s
19:	learn: 0.388696

In [ ]:
test_pred = pd.merge(
    left=pd.merge(
        left=test_warm_df.drop('real', axis=1).explode('recs').rename(columns={'recs': ITEM_COL}),
        right=data.items,
        on=[ITEM_COL],
        how='left'
    ),
    right=data.users,
    on=[USER_COL],
    how='left'
)

test_pred['sex'] = test_pred['sex'].fillna('unknown')
test_pred['age'] = test_pred['age'].fillna('age_unknown')
test_pred['genres'] = test_pred['genres'].fillna('genres_unknown')

test_pred['rating'] = cbc.predict_proba(test_pred[['sex', 'age', 'release_year_cat', 'content_type', 'genres']])[:, 1]
test_pred = test_pred.groupby(USER_COL).apply(lambda x: x.sort_values('rating', ascending=False)[ITEM_COL].tolist()[:10])

In [ ]:
map_at_k(
    10,
    test_pred.loc[test_warm_df[USER_COL].tolist()],
    test_warm_df['real']
)

In [60]:
test_warm_df['perc'] = test_warm_df.apply(lambda x: len(list(set(x['real']).intersection(set(x['recs'])))) / len(x['real']), axis=1)
test_warm_df['perc'].mean(), test_warm_df['perc'].quantile(0.25)

(0.3107621468081101, 0.0)